In [5]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [8]:
# global variables
start_features = ["Age", "Department"]
target = "Attrition"



In [101]:
def main():
    # create dataframe with project's data set
    df = pd.read_csv('HR.csv')
    # clean the data
    clean_df = clean_data(df)
    # get features matrix X and target variable y
    X, y = feature_selection(clean_df)
    

In [12]:
def extract_categoricals(df):
    # extract a list of all categorical features for feature engineering
    cat_features = df.select_dtypes(include=['object']).columns.tolist()
    return cat_features
    
        

In [14]:
def clean_data(df):
    # drop columns that are not needed
    for col_header in df.columns:
        if col_header not in start_features and col_header != target:
            df.drop(col_header, axis = 1, inplace=True)         
    # drop rows that contain at least one empty cell
    df.dropna(axis='rows', thresh=1)
    # change Yes or No in Attrition column to 0 and 1
    df[target] = df[target].map({'Yes': 1, 'No': 0})
    # feature enigneering for categorical data
    df = pd.get_dummies(data=df, columns=extract_categoricals(df))   
    return df


In [23]:
def feature_selection(df):
    # get new feature list
    features = df.columns.tolist()
    # exclude target from features
    features.pop(features.index(target))
    # split dataset in features and target variable
    X = df[features] # Features Matrix
    y = df[target] # Target variable
    return X, y


In [110]:
def construct_tree(X,y):
    decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
    decision_tree = decision_tree.fit(X.data, y.target)
    tree.plot_tree(decision_tree)

In [24]:
# test
df = pd.read_csv('HR.csv')
# clean the data
clean_df = clean_data(df)

# get features matrix X and target variable y
X, y = feature_selection(clean_df)
print("X: ", X)
print("y: ", y)



X:        Age  Department_Human Resources  Department_Research & Development  \
0      41                           0                                  0   
1      49                           0                                  1   
2      37                           0                                  1   
3      33                           0                                  1   
4      27                           0                                  1   
...   ...                         ...                                ...   
1465   36                           0                                  1   
1466   39                           0                                  1   
1467   27                           0                                  1   
1468   49                           0                                  0   
1469   34                           0                                  1   

      Department_Sales  
0                    1  
1                    0  
2       